<a href="https://colab.research.google.com/github/louis-not/Notogo-ML/blob/main/model_v1.1/Copy_of_Recommender_System_(Retrieval)_Ver_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialize

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 1.5 MB/s 
     |████████████████████████████████| 462 kB 30.9 MB/s 
     |████████████████████████████████| 4.2 MB 5.8 MB/s 


In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

import os
import pandas as pd
import pprint

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# cd /content/drive/Othercomputers/'My Laptop'/Bangkit/Capstone/'Recommender system'

In [5]:
cd /content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/"model_v1.2"

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v1.2


In [6]:
print(os.getcwd())

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v1.2


## Model

### Import Data

In [7]:
import userFeatures
builder = tfds.builder('Userfeatures')
userFeatureDs = tfds.load('Userfeatures',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userfeatures/1.0.0.incompleteGAS90V/userfeatures-train.tfrecord*...:   0%|…

Dataset userfeatures downloaded and prepared to /root/tensorflow_datasets/userfeatures/1.0.0. Subsequent calls will reuse this data.


In [8]:
import wishEmbedding
builder = tfds.builder('Wishembedding')
wishEmbeddingDs = tfds.load('Wishembedding',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishembedding/1.0.0.incompleteEO9LXD/wishembedding-train.tfrecord*...:   0…

Dataset wishembedding downloaded and prepared to /root/tensorflow_datasets/wishembedding/1.0.0. Subsequent calls will reuse this data.


In [9]:
import wishFeatures
builder = tfds.builder('Wishfeatures')
wishFeaturesDs = tfds.load('Wishfeatures',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishfeatures/1.0.0.incomplete6B5EVC/wishfeatures-train.tfrecord*...:   0%|…

Dataset wishfeatures downloaded and prepared to /root/tensorflow_datasets/wishfeatures/1.0.0. Subsequent calls will reuse this data.


In [10]:
for x in userFeatureDs.take(5).as_numpy_iterator():
  pprint.pprint(x)

{'location_id': b'61', 'location_name': b'JAKARTA', 'user_id': b'93'}
{'location_id': b'53', 'location_name': b'DUBAI', 'user_id': b'112'}
{'location_id': b'2', 'location_name': b'RAJA AMPAT', 'user_id': b'6'}
{'location_id': b'28', 'location_name': b'TOKYO', 'user_id': b'152'}
{'location_id': b'19', 'location_name': b'LONDON', 'user_id': b'91'}


In [11]:
for x in wishEmbeddingDs.take(5).as_numpy_iterator():
  pprint.pprint(x)

{'location_id': b'39', 'location_name': b'PRANCIS'}
{'location_id': b'55', 'location_name': b'MEDAN'}
{'location_id': b'28', 'location_name': b'TOKYO'}
{'location_id': b'64', 'location_name': b'BALI TOWER'}
{'location_id': b'90', 'location_name': b'GEOPARK CILETUH'}


In [13]:
for x in wishFeaturesDs.take(5).as_numpy_iterator():
  pprint.pprint(x)

{'add_count': 92,
 'like_count': 268,
 'location_id': b'39',
 'location_name': b'PRANCIS',
 'rating': 0.25555557}
{'add_count': 84,
 'like_count': 175,
 'location_id': b'55',
 'location_name': b'MEDAN',
 'rating': 0.3243243}
{'add_count': 48,
 'like_count': 124,
 'location_id': b'28',
 'location_name': b'TOKYO',
 'rating': 0.27906978}
{'add_count': 21,
 'like_count': 66,
 'location_id': b'64',
 'location_name': b'BALI TOWER',
 'rating': 0.2413793}
{'add_count': 81,
 'like_count': 188,
 'location_id': b'90',
 'location_name': b'GEOPARK CILETUH',
 'rating': 0.30111524}


In [10]:
rating = userFeatureDs.map(lambda x: {
    "location_name": x["location_name"],
    "user_id": x["user_id"],
})
location = wishEmbeddingDs.map(lambda x: x["location_name"])

NameError: ignored

In [15]:
trainset_size = 0.8 * rating.__len__().numpy()
tf.random.set_seed(42)
shuffled = rating.shuffle(1000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(trainset_size)
test = shuffled.skip(trainset_size).take(rating.__len__().numpy() - trainset_size)

In [16]:
location_name = location.batch(1000)
user_ids = rating.batch(1000).map(lambda x: x["user_id"])

unique_location_name = np.unique(np.concatenate(list(location_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_location_name[:10]

array([b'AIR TERJUN SIPISO-PISO', b'AKIHABARA', b'ALASKA',
       b'AMERIKA SERIKAT', b'ANCOL', b'ASIA TIMUR', b'BALI',
       b'BALI TOWER', b'BANDUNG', b'BANGKA BELITUNG'], dtype=object)

### Implementing a model

Choosing the architecture of our model is a key part of modelling.

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

#### The query tower

In [30]:
# embedding_dimension = 32
embedding_dimension = 16

In [31]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [42]:
location_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_location_name, mask_token=None),
  tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension)
])

#### Metrics

In our training data we have positive (user, movie) pairs. To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

To do this, we can use the `tfrs.metrics.FactorizedTopK` metric. The metric has one required argument: the dataset of candidates that are used as implicit negatives for evaluation.

In our case, that's the `movies` dataset, converted into embeddings via our movie model:

In [33]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=location.batch(128).map(location_model)
)

In [34]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

#### Loss

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `Retrieval` task object: a convenience wrapper that bundles together the loss function and metric computation:

#### The full model

We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines building models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [35]:
class Model(tfrs.Model):

  def __init__(self, user_model, location_model):
    super().__init__()
    self.location_model: tf.keras.Model = location_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_location_embeddings = self.location_model(features["location_name"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_location_embeddings)

In [ ]:
# class NoBaseClassMovielensModel(tf.keras.Model):

#   def __init__(self, user_model, location_model):
#     super().__init__()
#     self.location_model: tf.keras.Model = location_model
#     self.user_model: tf.keras.Model = user_model
#     self.task: tf.keras.layers.Layer = task

#   def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

#     # Set up a gradient tape to record gradients.
#     with tf.GradientTape() as tape:

#       # Loss computation.
#       user_embeddings = self.user_model(features["user_id"])
#       # positive_movie_embeddings = self.movie_model(features["movie_title"])
#       positive_location_embeddings = self.location_model(features["location_name"])
#       loss = self.task(user_embeddings, positive_location_embeddings)

#       # Handle regularization losses as well.
#       regularization_loss = sum(self.losses)

#       total_loss = loss + regularization_loss

#     gradients = tape.gradient(total_loss, self.trainable_variables)
#     self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

#     metrics = {metric.name: metric.result() for metric in self.metrics}
#     metrics["loss"] = loss
#     metrics["regularization_loss"] = regularization_loss
#     metrics["total_loss"] = total_loss

#     return metrics

#   def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

#     # Loss computation.
#     user_embeddings = self.user_model(features["user_id"])
#     # positive_movie_embeddings = self.movie_model(features["movie_title"])
#     positive_location_embeddings = self.location_model(features["location_name"])
#     loss = self.task(user_embeddings, positive_location_embeddings)

#     # Handle regularization losses as well.
#     regularization_loss = sum(self.losses)

#     total_loss = loss + regularization_loss

#     metrics = {metric.name: metric.result() for metric in self.metrics}
#     metrics["loss"] = loss
#     metrics["regularization_loss"] = regularization_loss
#     metrics["total_loss"] = total_loss

#     return metrics

### Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [36]:
model = Model(user_model, location_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001))

In [37]:
cached_train = train.shuffle(1000).batch(256).cache()
cached_test = test.batch(1000).cache()

In [38]:
model.fit(cached_train, epochs=40)

Epoch 1/40
86/86 [==============================] - 9s 83ms/step - factorized_top_k/top_1_categorical_accuracy: 1.3661e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0255 - factorized_top_k/top_10_categorical_accuracy: 0.0585 - factorized_top_k/top_50_categorical_accuracy: 0.3689 - factorized_top_k/top_100_categorical_accuracy: 0.7953 - loss: 1412.7392 - regularization_loss: 0.0000e+00 - total_loss: 1412.7392
Epoch 2/40
86/86 [==============================] - 6s 75ms/step - factorized_top_k/top_1_categorical_accuracy: 1.3661e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0343 - factorized_top_k/top_10_categorical_accuracy: 0.0777 - factorized_top_k/top_50_categorical_accuracy: 0.4332 - factorized_top_k/top_100_categorical_accuracy: 0.8454 - loss: 1410.4882 - regularization_loss: 0.0000e+00 - total_loss: 1410.4882
Epoch 3/40
86/86 [==============================] - 5s 62ms/step - factorized_top_k/top_1_categorical_accuracy: 2.2769e-04 - factorized_top_k/top_5_categorical_ac

In [39]:
model.evaluate(cached_test, return_dict=True)

6/6 [==============================] - 2s 107ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0126 - factorized_top_k/top_10_categorical_accuracy: 0.0308 - factorized_top_k/top_50_categorical_accuracy: 0.2933 - factorized_top_k/top_100_categorical_accuracy: 0.7672 - loss: 5986.3141 - regularization_loss: 0.0000e+00 - total_loss: 5986.3141


{'factorized_top_k/top_100_categorical_accuracy': 0.7672131061553955,
 'factorized_top_k/top_10_categorical_accuracy': 0.030783241614699364,
 'factorized_top_k/top_1_categorical_accuracy': 0.0010928962146863341,
 'factorized_top_k/top_50_categorical_accuracy': 0.29326048493385315,
 'factorized_top_k/top_5_categorical_accuracy': 0.012568306177854538,
 'loss': 3143.651123046875,
 'regularization_loss': 0,
 'total_loss': 3143.651123046875}

In [40]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((location.batch(100), location.batch(100).map(model.location_model)))
)

# Get recommendations.
user_id = "5"
_, titles = index(tf.constant([user_id]))
print(f"Recommendations for Rolas : {titles[0, :10]}")

Recommendations for Rolas : [b'LAMPUNG' b'NUSAWIRU PARACENTER' b'TOKYO' b'NEW YORK' b'GUNUNG'
 b'SWITZERLAND' b'DANAU SALAR DE UYUNI' b'PANGANDARAN' b'GUNUNG PANGRANGO'
 b'CANDI PRAMBANAN']


Recommendations for Louis: [b'Branchsto BSD' b'Eropa' b'Ponorogo' b'europe'
 b'Museum Nasional ImersifA' b'Gunung' b'Sumbar' b'Gunung Semeru'
 b'Maldives' b'Malang']